<a href="https://colab.research.google.com/github/SPARSHTHALYARI/Churn_detection-/blob/main/Chrun_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import numpy as np



In [ ]:
data=pd.read_csv('/content/customer_churn_large_dataset.csv')

data.head()

data.describe()

,CustomerID,Age,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB,Churn
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,50000.500000,44.027020,12.490100,65.053197,274.393650,0.497790
std,28867.657797,15.280283,6.926461,20.230696,130.463063,0.499998
min,1.000000,18.000000,1.000000,30.000000,50.000000,0.000000
25%,25000.750000,31.000000,6.000000,47.540000,161.000000,0.000000
50%,50000.500000,44.000000,12.000000,65.010000,274.000000,0.000000
75%,75000.250000,57.000000,19.000000,82.640000,387.000000,1.000000
max,100000.000000,70.000000,24.000000,100.000000,500.000000,1.000000


In [ ]:
data.info()
data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   CustomerID                  100000 non-null  int64  
 1   Name                        100000 non-null  object 
 2   Age                         100000 non-null  int64  
 3   Gender                      100000 non-null  object 
 4   Location                    100000 non-null  object 
 5   Subscription_Length_Months  100000 non-null  int64  
 6   Monthly_Bill                100000 non-null  float64
 7   Total_Usage_GB              100000 non-null  int64  
 8   Churn                       100000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 6.9+ MB


CustomerID                    0
Name                          0
Age                           0
Gender                        0
Location                      0
Subscription_Length_Months    0
Monthly_Bill                  0
Total_Usage_GB                0
Churn                         0
dtype: int64

In [ ]:
#he interpolate() function uses existing values in the DataFrame to estimate the missing rows. Setting the inplace keyword to True alters the DataFrame permanently.



data.interpolate(method ='linear', limit_direction ='forward', inplace=True)
data.isna().sum()

CustomerID                    0
Name                          0
Age                           0
Gender                        0
Location                      0
Subscription_Length_Months    0
Monthly_Bill                  0
Total_Usage_GB                0
Churn                         0
dtype: int64

In [ ]:


from sklearn.preprocessing import StandardScaler

# Assuming your DataFrame is df and the age column is 'Age'
scaler = StandardScaler()
data['Age'] = scaler.fit_transform(data[['Age']])

In [ ]:


# Assuming your DataFrame is named 'df'

# Perform one-hot encoding for the 'Gender' column
data = pd.get_dummies(data, columns=['Gender'], prefix=['Gender'])

# Perform one-hot encoding for the 'Location' column
data = pd.get_dummies(data, columns=['Location'], prefix=['Location'])

# Optional: If 'Name' is categorical and you want to one-hot encode it
data = pd.get_dummies(data, columns=['Name'], prefix=['Name'])

USING RANDOM FOREST FOR OUR MODEL

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming your DataFrame is named 'data'

# Step 1: Select features and target variable
features = ['Age',  'Subscription_Length_Months', 'Monthly_Bill', 'Total_Usage_GB']

target = 'Churn'  # Assuming 'Churn' is your target variable

X = data[features]
y = data[target]

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create and train a Random Forest classifier
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# Step 4: Make predictions on the test set
y_pred_rf = random_forest_model.predict(X_test)

# Step 5: Evaluate model performance
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.5018400802944128
Random Forest Classification Report:
               precision    recall  f1-score   support

         0.0       0.50      0.52      0.51      6002
         1.0       0.50      0.49      0.49      5954

    accuracy                           0.50     11956
   macro avg       0.50      0.50      0.50     11956
weighted avg       0.50      0.50      0.50     11956



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming your DataFrame is named 'data'

# Step 1: Feature Engineering
# 1. Age Group Binning
age_bins = [0, 18, 35, 60, float('inf')]
age_labels = ['<18', '18-35', '36-60', '60+']
data['Age_Group'] = pd.cut(data['Age'], bins=age_bins, labels=age_labels)

# 2. Bill to Usage Ratio
data['Bill_Per_GB'] = data['Monthly_Bill'] / data['Total_Usage_GB']

# 3. Subscription Length Category
subscription_bins = [0, 6, 12, 24, float('inf')]
subscription_labels = ['0-6 months', '7-12 months', '13-24 months', '24+ months']
data['Subscription_Length_Category'] = pd.cut(data['Subscription_Length_Months'], bins=subscription_bins, labels=subscription_labels)

# Step 2: Select features and target variable
features = ['Age_Group', 'Subscription_Length_Category', 'Bill_Per_GB', 'Total_Usage_GB']

target = 'Churn'  # Assuming 'Churn' is your target variable

X = data[features]
y = data[target]

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Create and train a Random Forest classifier
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# Step 5: Make predictions on the test set
y_pred_rf = random_forest_model.predict(X_test)

# Step 6: Evaluate model performance
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))


ValueError: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming your DataFrame is named 'data'

# Step 1: Feature Engineering
# 1. Age Group Binning
age_bins = [0, 18, 35, 60, float('inf')]
age_labels = ['<18', '18-35', '36-60', '60+']
data['Age_Group'] = pd.cut(data['Age'], bins=age_bins, labels=age_labels)

# 2. Bill to Usage Ratio
data['Bill_Per_GB'] = data['Monthly_Bill'] / data['Total_Usage_GB']

# 3. Subscription Length Category
subscription_bins = [0, 6, 12, 24, float('inf')]
subscription_labels = ['0-6 months', '7-12 months', '13-24 months', '24+ months']
data['Subscription_Length_Category'] = pd.cut(data['Subscription_Length_Months'], bins=subscription_bins, labels=subscription_labels)

# Step 2: Perform One-Hot Encoding for Categorical Features
data = pd.get_dummies(data, columns=['Age_Group', 'Subscription_Length_Category'], drop_first=True)

# Step 3: Select features and target variable
features = ['Bill_Per_GB', 'Total_Usage_GB', 'Age_Group_18-35', 'Age_Group_36-60', 'Age_Group_60+',
            'Subscription_Length_Category_7-12 months', 'Subscription_Length_Category_13-24 months', 'Subscription_Length_Category_24+ months']

target = 'Churn'  # Assuming 'Churn' is your target variable

X = data[features]
y = data[target]

# Step 4: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Create and train a Random Forest classifier
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# Step 6: Make predictions on the test set
y_pred_rf = random_forest_model.predict(X_test)

# Step 7: Evaluate model performance
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))



NameError: ignored